In [ ]:
#!pip install odc-algo odc-stac pystac-client python-dotenv coiled

In [ ]:
import os
os.environ["AWS_REQUEST_PAYER"] = "requester" 


In [ ]:
#%load_ext dotenv
#%dotenv

# logging 
import logging
logging.basicConfig()

# set pystac_client logger to DEBUG to see API calls
logger = logging.getLogger('pystac_client')
logger.setLevel(logging.INFO)

In [ ]:
import hvplot.pandas
import hvplot.xarray

# plot size settings
frame_width = 600
frame_height = 600

# line width of polygons
line_width = 3

# plot polygons as lines on a slippy map with background tiles.
def plot_polygons(data, *args, **kwargs):
    return data.hvplot.paths(*args, geo=True, tiles='OSM', xaxis=None, yaxis=None,
                             frame_width=frame_width, frame_height=frame_height,
                             line_width=line_width, **kwargs)

from copy import deepcopy
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape

# convert a list of STAC Items into a GeoDataFrame
def items_to_geodataframe(items):
    _items = []
    for i in items:
        _i = deepcopy(i)
        _i['geometry'] = shape(_i['geometry'])
        _items.append(_i)
    gdf = gpd.GeoDataFrame(pd.json_normalize(_items))
    for field in ['properties.datetime', 'properties.created', 'properties.updated']:
        if field in gdf:
            gdf[field] = pd.to_datetime(gdf[field])
    gdf.set_index('properties.datetime', inplace=True)
    return gdf

# AOI
We first start with a spatial AOI, which should be a single GeoJSON Feature with a geometry type of Point, LineString, Polygon, MultiPoint, MultiLineString, or MultiPolygon. A fast and simple tool to create such as an AOI is http://geojson.io/. Save a GeoJSON Feature (not a FeatureCollection!) in a file accessible by this notebook.

In [ ]:
aoi_filename = "./lakota.geojson"

# read in AOI as a GeoDataFrame
import geopandas as gpd
aoi = gpd.read_file(aoi_filename)

# get the geometry of the AOI as a dictionary for use with PySTAC Client
from shapely.geometry import mapping
geom = mapping(aoi.to_dict()['geometry'][0])

In [ ]:
geom

# Searching a STAC API
We can search any STAC API by providing the root URL to the Client.open() function.We can now use the AOI geometry for the intersects parameter with **pystac-client**. Edit/add parameters below.

In [ ]:
# STAC API - Landsat Collection 2
url = "https://landsatlook.usgs.gov/stac-server"

# Search parameters
params = {
    "collections": ["landsat-c2l2-sr"],
    "intersects": geom,
    "datetime": "2015-01-01/2020-12-31",
    "limit": 100,
    "query": ["platform=LANDSAT_8", "eo:cloud_cover<10"]
}

from pystac_client import Client
cat = Client.open(url)
search = cat.search(**params)

matched = search.matched()
print(f"{search.matched()} scenes found")

In [ ]:
#! conda install -y pystac-client

In [ ]:
%%time
from pystac import ItemCollection

# get all items found in search
items_dict = []
for item in search.get_all_items_as_dict()['features']:
    for a in item['assets']:
        if 'alternate' in item['assets'][a] and 's3' in item['assets'][a]['alternate']:
            item['assets'][a]['href'] = item['assets'][a]['alternate']['s3']['href']
        item['assets'][a]['href'] = item['assets'][a]['href'].replace('usgs-landsat-ard', 'usgs-landsat')
    items_dict.append(item)

# Create GeoDataFrame from resulting Items
items_gdf = items_to_geodataframe(items_dict)
item_collection = ItemCollection(items_dict)

# Using Pandas/GeoPandas
With the STAC Items in a GeoDataFrame, we can view the geometries of the Items GeoDataFrame and the AOI GeoDataFrame together.

In [ ]:
plot_polygons(items_gdf) * aoi.hvplot.paths(geo=True, line_width=3, line_color='red')

# STAC Assets
We haven't looked at any data yet, as the actual data in STAC Items is stored in assets so we are limited to exploring the metadata about the data. To use the assets a user should first understand what assets are available.  Here we use Pandas to take a look at the Asset dictionary of the first Item (note that not all Items may have the same assets).

In [ ]:
# use the first Item to see what assets are available

assets = pd.DataFrame.from_dict(items_dict[0]['assets'], orient='index')

for f in ['alternate', 'file:checksum', 'proj:transform', 'rel']:
    if f in assets:
        del assets[f]

assets

# OpenDataCube



In [ ]:
# Convert the STAC item(s) to ODC datasets
import yaml

from odc import stac
from pyproj import CRS
from pystac.extensions.projection import ProjectionExtension

def open_odc(items, crs=None, resolution=None):
    configuration_str = """---
        landsat-c2l2-sr:
          measurements:
            'red':
              dtype: float32
              nodata: 0
              units: 'm'
        """
    configuration = yaml.load(configuration_str, Loader=yaml.CSafeLoader)
    datasets = list(stac.stac2ds(items, configuration))
    
    proj = ProjectionExtension.ext(items[0])
    if crs is None:
        crs = CRS.from_epsg(proj.epsg)
    if resolution is None:
        resolution = (proj.transform[4], proj.transform[0])

    #data = stac.dc_load(datasets, measurements=['red','green','blue', 'nir08', 'qa_pixel'], chunks={"x": 1024, "y": 1024}, output_crs=crs, resolution=resolution)
    data = stac.dc_load(datasets, measurements=['red','green','blue', 'nir08', ], 
      chunks={"x": 1024, "y": 1024}, output_crs=crs, resolution=resolution)
    return data

In [ ]:
help(stac.stac2ds)

In [ ]:
# hello

In [ ]:
# open found items as an OpenDataCube

_datacube = open_odc(item_collection) #, 'epsg:4326', resolution='0.002')
_datacube.to_array(dim='bands')

In [ ]:
import rioxarray

datacube = _datacube.rio.clip([geom], crs='epsg:4326')
datacube.to_array(dim='bands')

In [ ]:
# create RGB image for display
from odc.algo import to_rgba

rgba = to_rgba(datacube, clamp=(1000,20000), bands=['red', 'green', 'blue'])
rgba

In [ ]:
from dask.distributed import Client, progress
client = Client(processes=False, threads_per_worker=8,
                n_workers=2, memory_limit='8GB', dashboard_address=':8080')
client

In [ ]:
rgba.compute()

In [ ]:
rgba

In [ ]:
rgba.hvplot.rgb('x', 'y', bands='band', groupby='time', aspect=1, frame_width=400)

In [ ]:
datacube['red'][1].plot()

In [ ]:
datacube['green'][10].plot(cmap='BuPu')

In [ ]:
# %%time
# # process with dask

# from dask.distributed import wait

# rgba = cluster.persist(rgba)
# _ = wait(rgba)
# rgba

In [ ]:
#help (Client.persist)

In [ ]:
datacube